In [1]:
!wget https://raw.githubusercontent.com/Dinarque/INALCO_Techniques_web_en_python_2025/refs/heads/main/TP/papyrus_corpus.csv

--2025-12-22 23:21:24--  https://raw.githubusercontent.com/Dinarque/INALCO_Techniques_web_en_python_2025/refs/heads/main/TP/papyrus_corpus.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2089431 (2.0M) [text/plain]
Saving to: ‘papyrus_corpus.csv’

papyrus_corpus.csv  100%[===================>]   1.99M  --.-KB/s    in 0.04s   

2025-12-22 23:21:24 (44.6 MB/s) - ‘papyrus_corpus.csv’ saved [2089431/2089431]



In [2]:
import polars as pl

df = pl.read_csv("/content/papyrus_corpus.csv", infer_schema_length=2000)

# Regex définissant les caractères grecs
# \u0370-\u03FF : Grec et Copte
# \u1F00-\u1FFF : Grec étendu (polytonique)
# \u0300-\u036F : Diacritiques combinants
greek_regex = r"[\u0370-\u03FF\u1F00-\u1FFF\u0300-\u036F]"

df = df.filter(
    # 1. La base : pas null et pas vide
    pl.col("Full Text").is_not_null() &
    (pl.col("Full Text").cast(pl.Utf8).str.strip_chars() != "") &

    # 2. Supprimer explicitement "nan" et "none" (votre logique originale)
    ~pl.col("Full Text")
      .cast(pl.Utf8)
      .str.to_lowercase()
      .str.contains("nan|none") &

    # 3. LE FILTRE MANQUANT : La ligne DOIT contenir au moins un caractère grec
    # C'est ce filtre qui fait passer de 459 à 456
    pl.col("Full Text").str.contains(greek_regex)
)

print(f"Nombre de lignes restantes : {len(df)}")

Nombre de lignes restantes : 456


In [3]:
import polars as pl

# Chargement
df = pl.read_csv("/content/papyrus_corpus.csv", infer_schema_length=2000)

# Regex pour le grec (la même que celle de votre script de nettoyage)
greek_regex = r"[\u0370-\u03FF\u1F00-\u1FFF\u0300-\u036F]"

# 1. Définition du filtre "Base" (Ce qui donnait 459 lignes)
# -> Pas vide, pas null, et ne contient pas "nan" ou "none"
filter_valid_text = (
    pl.col("Full Text").is_not_null() &
    (pl.col("Full Text").cast(pl.Utf8).str.strip_chars() != "") &
    ~pl.col("Full Text").cast(pl.Utf8).str.to_lowercase().str.contains("nan|none")
)

# 2. Définition du filtre "Grec"
# -> Contient des caractères grecs
filter_has_greek = pl.col("Full Text").str.contains(greek_regex)

# 3. La différence : Lignes valides (texte) MAIS sans grec
diff_df = df.filter(filter_valid_text & ~filter_has_greek)

print(f"Lignes valides textuellement (sans 'nan'/'none') : {len(df.filter(filter_valid_text))} (devrait être 459)")
print(f"Lignes contenant du grec : {len(df.filter(filter_valid_text & filter_has_greek))} (devrait être 456)")
print(f"Différence (Lignes sans grec) : {len(diff_df)}")
print("-" * 60)
print("VOICI LES LIGNES EXCLUES (Contenu non-vide, mais sans grec) :")
with pl.Config(fmt_str_lengths=1000): # Pour afficher tout le texte
    print(diff_df.select("Full Text"))

Lignes valides textuellement (sans 'nan'/'none') : 459 (devrait être 459)
Lignes contenant du grec : 456 (devrait être 456)
Différence (Lignes sans grec) : 3
------------------------------------------------------------
VOICI LES LIGNES EXCLUES (Contenu non-vide, mais sans grec) :
shape: (3, 1)
┌─────────────────────────────────────────┐
│ Full Text                               │
│ ---                                     │
│ str                                     │
╞═════════════════════════════════════════╡
│ 1 |gap=6_lines|                         │
│ 1 |gap=3_lines|                         │
│ 1 [--------------------]  |gap=2_lines| │
└─────────────────────────────────────────┘
